# First we will convert an Explainable SageMaker Image Classification Model

In [1]:
# Import some standard dependencies
import os
import json

## 1. Train a built-in model on the SageMaker Platform

The raw output of the SageMaker Platform will be a set of weights for your image classification model that is stored within a bucket on AWS S3.

In [2]:
# TODO: Include concise training code to showcase how the assets below are produced

blob_storage_provider = "S3"
blob_storage_container = "modzy-engineering-tests"
weights_key = "ds/model-converter/sagemaker/image-classification/weights.tar.gz"

## 2. Provide metadata specific to your model

You are encouraged to fill out a `model.yaml` file describing your model and provide any additional metadata files for the specific model type that you chose. In the case of image classification, you will need to provide a `labels.json` file that contains a mapping between numerical classes and human readable labels. For example, 12 could be the label for a "tabby cat".

In [3]:
from modzy.converter.utils import upload_resources

current_working_directory = os.getcwd()

# Indicate where your local files
model_yaml_path = os.path.join(current_working_directory, "examples/sagemaker/image-classification/model.yaml")
labels_json_path = os.path.join(current_working_directory, "examples/sagemaker/image-classification/labels.json")
auxiliary_files = [labels_json_path]

# Your local files will be archieved and stored in the following location:
resources_key = "ds/demo/model-converter/sagemaker/image-classification/resources.tar.gz"


upload_resources(
    model_yaml_path, blob_storage_container, resources_key,
    os.getenv("SP_ACCESS_KEY_ID"), os.getenv("SP_SECRET_ACCESS_KEY"), blob_storage_provider, auxiliary_files
)

## 3. Use the Modzy Python SDK to set up an API Client to the Model Converter Service

The Modzy Python SDK has a number of convenience functions that we will use to:
* connect to our instance of modzy
* interact with the model converter service
* manage processing engines for our new model
* run jobs against our new model

In [4]:
from modzy.converter.model_converter import ModelConverter
from modzy.client import ApiClient

# To get started, store your Modzy API key as an environment variable `MODZY_API_KEY`.
# Then, create a Modzy API client to interact with the integration envrionment
modzy_api_key = os.getenv("MODZY_QA_API_KEY")
modzy_instance_base_url = "https://integration.modzy.engineering/api"
modzy_api_client = ApiClient(api_key=modzy_api_key, base_url=modzy_instance_base_url)

# Instantiate a Model Converter client with access to the Modzy integration environment
model_converter = ModelConverter(modzy_api_client)

In [ ]:
# Now, provide the Model converter with information about your stored model assets and the credentials required
# to access them. The Model converter will do the rest of the work.

source_platform = "sagemaker"
model_type = "image-classification"

_, sagemaker_converter_output = model_converter.run(
        sp_access_key_id=os.getenv("SP_ACCESS_KEY_ID"),
        sp_secret_access_key=os.getenv("SP_SECRET_ACCESS_KEY"),
        blobstore_provider=blob_storage_provider,
        blobstore_container=blob_storage_container,
        weights_path=weights_key,
        resources_path=resources_key,
        platform=source_platform,
        model_type=model_type,
)

print(f"The model details page for your new model can be found here: {sagemaker_converter_output['modelURL']}")
sagemaker_model_id = sagemaker_converter_output["modelId"]
sagemaker_model_version = sagemaker_converter_output["modelVersion"]

In [ ]:
# Delegate a single processing engine to serve your new model
modzy_api_client.models.update_processing_engines(
    sagemaker_model_id, sagemaker_model_version, min_engines=1, max_engines=1
)
print(
    f"Warming up a container to perform inference from model {sagemaker_model_id} version {sagemaker_model_version}"
)

In [ ]:
# Send an inference job to run against your new model with explainability!
sagemaker_input_source = {
    "00001": {
        f"image": {
            "bucket": blob_storage_container,
            "key": f"/ds/model-converter/{source_platform}/{model_type}/test_input"
        }
    }
}

print(f"Sending job to model {sagemaker_model_id} version {sagemaker_model_version}")
job = modzy_api_client.jobs.submit_aws_s3_bulk(
    sagemaker_model_id, sagemaker_model_version, sagemaker_input_source,
    os.getenv("SP_ACCESS_KEY_ID"), os.getenv("SP_SECRET_ACCESS_KEY"),
    region="us-east-1", explain=True
)

modzy_api_client.jobs.block_until_complete(job, timeout=None)
print("Job Completed!")

In [ ]:
# Release the processing engine
modzy_api_client.models.update_processing_engines(
    sagemaker_model_id, sagemaker_model_version, min_engines=0, max_engines=1
)

# Next, we will convert an MLflow model

In [ ]:
from modzy.converter.mlflow import upload_mlflow_model

# Now we repeat the process with an MLFlow model
source_platform = "mlflow"
model_type = "tabular"

mlflow_model_dir = ""
mlflow_weights_key = "ds/demo/model-converter/sagemaker/mlflow/tabular/weights.tar.gz"

# upload_mlflow_model(
#     mlflow_model_dir, blob_storage_container, mlflow_weights_key,
#     os.getenv("SP_ACCESS_KEY_ID"), os.getenv("SP_SECRET_ACCESS_KEY"), blob_storage_provider
# )

mlflow_model_yaml_path = ""
mlflow_resources_key = "ds/demo/model-converter/sagemaker/mlflow/tabular/resources.tar.gz"

# upload_resources(
#     mlflow_model_yaml_path, blob_storage_container, mlflow_resources_key,
#     os.getenv("SP_ACCESS_KEY_ID"), os.getenv("SP_SECRET_ACCESS_KEY"), blob_storage_provider, auxiliary_files
# )

In [ ]:
_, mlflow_converter_output = model_converter.run(
        sp_access_key_id=os.getenv("SP_ACCESS_KEY_ID"),
        sp_secret_access_key=os.getenv("SP_SECRET_ACCESS_KEY"),
        blobstore_provider=blob_storage_provider,
        blobstore_container=blob_storage_container,
        weights_path=weights_key,
        resources_path=resources_key,
        platform=source_platform,
        model_type=model_type,
)

print(f"The model details page for your new model can be found here: {mlflow_converter_output['modelURL']}")
mlflow_model_id = mlflow_converter_output["modelId"]
mlflow_model_version = mlflow_converter_output["modelVersion"]

In [ ]:
# Delegate a single processing to serve your new model
modzy_api_client.models.update_processing_engines(mlflow_model_id, mlflow_model_version, min_engines=1, max_engines=1)

In [ ]:
# Send an inference job to run against your new model!
input_source = {
    "0001": {
        f"input.csv": {
            "bucket": blob_storage_container,
            "key": f"/ds/model-converter/{source_platform}/{model_type}/test_input"
        }
    }
}

print(f"Sending job to model {mlflow_model_id} {mlflow_model_version}")
job = modzy_api_client.jobs.submit_aws_s3_bulk(
    mlflow_model_id, mlflow_model_version, input_source,
    os.getenv("SP_ACCESS_KEY_ID"), os.getenv("SP_SECRET_ACCESS_KEY"),
    region="us-east-1"
)

modzy_api_client.jobs.block_until_complete(job, timeout=None)
print("Job Completed!")

In [ ]:
# Spin down the resources when down performing inference
modzy_api_client.models.update_processing_engines(mlflow_model_id, mlflow_model_version, min_engines=0, max_engines=1)